_한국어로 기계번역됨_


# 도구 내에서 데이터 스트리밍하는 방법

!!! 정보 "전제 조건"

    이 가이드는 다음에 대한 충분한 이해를 가정합니다:
    
    - [스트리밍](../../concepts/streaming/)
    - [채팅 모델](https://python.langchain.com/docs/concepts/chat_models/)
    - [도구](https://python.langchain.com/docs/concepts/tools/)

그래프가 LLM이나 기타 스트리밍 API를 사용하는 도구를 호출하는 경우, 도구의 실행 중에 부분 결과를 표시할 수 있습니다. 특히 도구 실행 시간이 길 경우 유용합니다.

1. 도구 내에서 **임의의** 데이터를 스트리밍하려면 [`stream_mode="custom"`](../streaming#custom) 및 `get_stream_writer()`를 사용할 수 있습니다:

    ```python
    # 다음 줄 강조
    from langgraph.config import get_stream_writer
    
    def tool(tool_arg: str):
        writer = get_stream_writer()
        for chunk in custom_data_stream():
            # 임의의 데이터 스트리밍
            # 다음 줄 강조
            writer(chunk)
        ...
    
    for chunk in graph.stream(
        inputs,
        # 다음 줄 강조
        stream_mode="custom"
    ):
        print(chunk)
    ```

2. LLM을 호출하는 도구에서 생성된 LLM 토큰을 스트리밍하려면 [`stream_mode="messages"`](../streaming#messages)를 사용할 수 있습니다:

    ```python
    from langgraph.graph import StateGraph, MessagesState
    from langchain_openai import ChatOpenAI
    
    model = ChatOpenAI()
    
    def tool(tool_arg: str):
        model.invoke(tool_arg)
        ...
    
    def call_tools(state: MessagesState):
        tool_call = get_tool_call(state)
        tool_result = tool(**tool_call["args"])
        ...
    
    graph = (
        StateGraph(MessagesState)
        .add_node(call_tools)
        ...
        .compile()
    
    for msg, metadata in graph.stream(
        inputs,
        # 다음 줄 강조
        stream_mode="messages"
    ):
        print(msg)
    ```

!!! 노트 "LangChain 없이 사용"

    **LangChain을 사용하지 않고** 도구 내에서 데이터를 스트리밍해야 하는 경우, [`stream_mode="custom"`](../streaming/#custom)를 사용할 수 있습니다. 자세한 내용은 [아래 예제](#example-without-langchain)를 확인하세요.

!!! 경고 "Python < 3.11에서의 비동기 처리"
    
    Python < 3.11에서 비동기 코드를 사용할 때는 `RunnableConfig`를 수동으로 채팅 모델에 전달해야 합니다. 호출할 때 다음과 같이 전달합니다: `model.ainvoke(..., config)`. 스트림 메서드는 콜백으로 전달된 스트리밍 트레이서를 사용하여 중첩된 코드의 모든 이벤트를 수집합니다. 3.11 이상에서는 이것이 자동으로 처리되지만, 3.11 이전의 [asyncio의 작업](https://docs.python.org/3/library/asyncio-task.html#asyncio.create_task)은 적절한 `contextvar` 지원이 부족하여 설정을 수동으로 전달해야만 콜백이 전파됩니다. 우리는 아래의 `call_model` 함수에서 이를 처리합니다.

## 설정

먼저, 필요한 패키지를 설치하고 API 키를 설정해 보겠습니다.


In [1]:
%%capture --no-stderr
%pip install -U langgraph langchain-openai


In [2]:
import getpass
import os


def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")


_set_env("OPENAI_API_KEY")


OPENAI_API_KEY:  ········


<div class="admonition tip">
    <p class="admonition-title">LangGraph 개발을 위한 <a href="https://smith.langchain.com">LangSmith</a> 설정</p>
    <p style="padding-top: 5px;">
        LangSmith에 가입하여 LangGraph 프로젝트의 문제를 신속하게 파악하고 성능을 개선하세요. LangSmith는 추적 데이터를 사용하여 LangGraph로 구축된 LLM 앱을 디버깅하고 테스트하며 모니터링할 수 있게 해줍니다 — 시작하는 방법에 대한 자세한 내용은 <a href="https://docs.smith.langchain.com">여기</a>를 읽어보세요.
    </p>
</div>


## 사용자 지정 데이터 스트리밍

이 가이드를 위해 [미리 구축된 ReAct 에이전트][langgraph.prebuilt.chat_agent_executor.create_react_agent]를 사용할 것입니다:


In [3]:
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI

from langgraph.prebuilt import create_react_agent
from langgraph.config import get_stream_writer


@tool
async def get_items(place: str) -> str:
    """Use this tool to list items one might find in a place you're asked about."""
    # highlight-next-line
    writer = get_stream_writer()

    # this can be replaced with any actual streaming logic that you might have
    items = ["books", "penciles", "pictures"]
    for chunk in items:
        # highlight-next-line
        writer({"custom_tool_data": chunk})

    return ", ".join(items)


llm = ChatOpenAI(model_name="gpt-4o-mini")
tools = [get_items]
# contains `agent` (tool-calling LLM) and `tools` (tool executor) nodes
agent = create_react_agent(llm, tools=tools)


이제 도구 호출이 필요한 입력으로 에이전트를 호출해 보겠습니다:


In [4]:
inputs = {
    "messages": [  # noqa
        {"role": "user", "content": "what items are in the office?"}
    ]
}
async for chunk in agent.astream(
    inputs,
    # highlight-next-line
    stream_mode="custom",
):
    print(chunk)


{'custom_tool_data': 'books'}
{'custom_tool_data': 'penciles'}
{'custom_tool_data': 'pictures'}


## 스트리밍 LLM 토큰


In [5]:
from langchain_core.messages import AIMessageChunk
from langchain_core.runnables import RunnableConfig


@tool
async def get_items(
    place: str,
    # Manually accept config (needed for Python <= 3.10)
    # highlight-next-line
    config: RunnableConfig,
) -> str:
    """Use this tool to list items one might find in a place you're asked about."""
    # Attention: when using async, you should be invoking the LLM using ainvoke!
    # If you fail to do so, streaming will NOT work.
    response = await llm.ainvoke(
        [
            {
                "role": "user",
                "content": (
                    f"Can you tell me what kind of items i might find in the following place: '{place}'. "
                    "List at least 3 such items separating them by a comma. And include a brief description of each item."
                ),
            }
        ],
        # highlight-next-line
        config,
    )
    return response.content


tools = [get_items]
# contains `agent` (tool-calling LLM) and `tools` (tool executor) nodes
agent = create_react_agent(llm, tools=tools)


In [6]:
inputs = {
    "messages": [  # noqa
        {"role": "user", "content": "what items are in the bedroom?"}
    ]
}
async for msg, metadata in agent.astream(
    inputs,
    # highlight-next-line
    stream_mode="messages",
):
    if (
        isinstance(msg, AIMessageChunk)
        and msg.content
        # Stream all messages from the tool node
        # highlight-next-line
        and metadata["langgraph_node"] == "tools"
    ):
        print(msg.content, end="|", flush=True)


Certainly|!| Here| are| three| items| you| might| find| in| a| bedroom|:

|1|.| **|Bed|**|:| The| central| piece| of| furniture| in| a| bedroom|,| typically| consisting| of| a| mattress| supported| by| a| frame|.| It| is| designed| for| sleeping| and| can| vary| in| size| from| twin| to| king|.| Beds| often| have| bedding|,| including| sheets|,| pillows|,| and| comfort|ers|,| to| enhance| comfort|.

|2|.| **|D|resser|**|:| A| piece| of| furniture| with| drawers| used| for| storing| clothing| and| personal| items|.| Dress|ers| often| have| a| flat| surface| on| top|,| which| can| be| used| for| decorative| items|,| a| mirror|,| or| personal| accessories|.| They| help| keep| the| bedroom| organized| and| clutter|-free|.

|3|.| **|Night|stand|**|:| A| small| table| or| cabinet| placed| beside| the| bed|,| used| for| holding| items| such| as| a| lamp|,| alarm| clock|,| books|,| or| personal| items|.| Night|stands| provide| convenience| for| easy| access| to| essentials| during| the| night|

## LangChain 없는 예시


도구 호출 내에서 **LangChain을 사용하지 않고** 데이터 스트리밍도 가능합니다. 아래 예제는 단일 도구 실행 노드를 가진 그래프에서 이를 수행하는 방법을 보여줍니다. LangChain을 사용하지 않고 [ReAct 에이전트를 처음부터 구현하는 것](../react-agent-from-scratch)은 독자에게 연습으로 남겨두겠습니다.


In [7]:
import operator
import json

from typing import TypedDict
from typing_extensions import Annotated
from langgraph.graph import StateGraph, START

from openai import AsyncOpenAI

openai_client = AsyncOpenAI()
model_name = "gpt-4o-mini"


async def stream_tokens(model_name: str, messages: list[dict]):
    response = await openai_client.chat.completions.create(
        messages=messages, model=model_name, stream=True
    )
    role = None
    async for chunk in response:
        delta = chunk.choices[0].delta

        if delta.role is not None:
            role = delta.role

        if delta.content:
            yield {"role": role, "content": delta.content}


# this is our tool
async def get_items(place: str) -> str:
    """Use this tool to list items one might find in a place you're asked about."""
    # highlight-next-line
    writer = get_stream_writer()
    response = ""
    async for msg_chunk in stream_tokens(
        model_name,
        [
            {
                "role": "user",
                "content": (
                    "Can you tell me what kind of items "
                    f"i might find in the following place: '{place}'. "
                    "List at least 3 such items separating them by a comma. "
                    "And include a brief description of each item."
                ),
            }
        ],
    ):
        response += msg_chunk["content"]
        # highlight-next-line
        writer(msg_chunk)

    return response


class State(TypedDict):
    messages: Annotated[list[dict], operator.add]


# this is the tool-calling graph node
async def call_tool(state: State):
    ai_message = state["messages"][-1]
    tool_call = ai_message["tool_calls"][-1]

    function_name = tool_call["function"]["name"]
    if function_name != "get_items":
        raise ValueError(f"Tool {function_name} not supported")

    function_arguments = tool_call["function"]["arguments"]
    arguments = json.loads(function_arguments)

    function_response = await get_items(**arguments)
    tool_message = {
        "tool_call_id": tool_call["id"],
        "role": "tool",
        "name": function_name,
        "content": function_response,
    }
    return {"messages": [tool_message]}


graph = (
    StateGraph(State)  # noqa
    .add_node(call_tool)
    .add_edge(START, "call_tool")
    .compile()
)


이제 도구 호출이 포함된 AI 메시지로 그래프를 호출해 봅시다:


In [8]:
inputs = {
    "messages": [
        {
            "content": None,
            "role": "assistant",
            "tool_calls": [
                {
                    "id": "1",
                    "function": {
                        "arguments": '{"place":"bedroom"}',
                        "name": "get_items",
                    },
                    "type": "function",
                }
            ],
        }
    ]
}

async for chunk in graph.astream(
    inputs,
    # highlight-next-line
    stream_mode="custom",
):
    print(chunk["content"], end="|", flush=True)


Sure|!| Here| are| three| common| items| you| might| find| in| a| bedroom|:

|1|.| **|Bed|**|:| The| focal| point| of| the| bedroom|,| a| bed| typically| consists| of| a| mattress| resting| on| a| frame|,| and| it| may| include| pillows| and| bedding|.| It| provides| a| comfortable| place| for| sleeping| and| resting|.

|2|.| **|D|resser|**|:| A| piece| of| furniture| with| multiple| drawers|,| a| dresser| is| used| for| storing| clothes|,| accessories|,| and| personal| items|.| It| often| has| a| flat| surface| that| may| be| used| to| display| decorative| items| or| a| mirror|.

|3|.| **|Night|stand|**|:| Also| known| as| a| bedside| table|,| a| night|stand| is| placed| next| to| the| bed| and| typically| holds| items| like| lamps|,| books|,| alarm| clocks|,| and| personal| belongings| for| convenience| during| the| night|.

|These| items| contribute| to| the| functionality| and| comfort| of| the| bedroom| environment|.|